In [1]:
import keras
from keras.applications.vgg16 import VGG16
#load model
model = VGG16()

Using TensorFlow backend.


In [2]:
model.summary()


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [3]:
from keras.preprocessing.image import load_img
from keras.applications.imagenet_utils import preprocess_input


In [4]:
from __future__ import print_function

import numpy as np
import warnings

from keras.models import Model
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalMaxPooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import preprocess_input
from keras.applications.imagenet_utils import _obtain_input_shape
from keras.utils.layer_utils import get_source_inputs

ImportError: cannot import name '_obtain_input_shape' from 'keras.applications.imagenet_utils' (C:\Users\ishno\Anaconda3\lib\site-packages\keras\applications\imagenet_utils.py)

In [5]:
def VGGupdated(input_tensor=None, classes = 4):
    img_rows, img_cols = 176,208 #default size 224,244
    img_channels = 3
    img_dim = (img_rows, img_cols, img_channels)
    
    img_input = Input(shape = img_dim)
    
    #Block1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)
    
    #Block2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)
    
    #Block3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)
    
    #Block4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)
    
    #Block5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    
    #Classification Block
    x = Flatten(name = 'flatten')(x)
    x = Dense(4906, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)
    
    #Create Model
    model = Model(inputs = img_input, outputs = x, name= 'VGGdemo')
    
    return model

In [6]:
model = VGGupdated(classes = 4)

In [7]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

dataset_path = os.listdir('Alzheimer_s Dataset')

class_type = os.listdir('Alzheimer_s Dataset')
print(type) 

print("Classes found: ", len(dataset_path))
                          

<class 'type'>
Classes found:  4


In [9]:
classes = []

for item in class_type:
    #Get all the file names
    all_classes = os.listdir('Alzheimer_s Dataset' + '/' +item)
    
    for type in all_classes:
        classes.append((item, str('Alzheimer_s Dataset' + '/' +item) + '/' + type))
        
        
                                

In [10]:
#Build a dataframe of images and labels
classes_df = pd.DataFrame(data = classes, columns=['class', 'image'])
print(classes_df.head())


          class                                         image
0  MildDemented  Alzheimer_s Dataset/MildDemented/26 (19).jpg
1  MildDemented  Alzheimer_s Dataset/MildDemented/26 (20).jpg
2  MildDemented  Alzheimer_s Dataset/MildDemented/26 (21).jpg
3  MildDemented  Alzheimer_s Dataset/MildDemented/26 (22).jpg
4  MildDemented  Alzheimer_s Dataset/MildDemented/26 (23).jpg


In [11]:
#Checking how many samples for each categories exist
print("Total number of samples in the dataset: ", len(classes_df))
class_count = classes_df['class'].value_counts()
print("Counts of samples in each category: ")
print(class_count)

Total number of samples in the dataset:  6400
Counts of samples in each category: 
NonDemented         3200
VeryMildDemented    2240
MildDemented         896
ModerateDemented      64
Name: class, dtype: int64


In [25]:
import cv2
path = 'Alzheimer_s dataset/'

img_width = 176
img_height = 208

images = []
labels = []

for i in class_type:
    data_path = path + str(i)
    filenames = [i for i in os.listdir(data_path)]
    
    
    for f in filenames:
        img = cv2.imread(data_path + '/' + f)
        img = cv2.resize(img, (img_width, img_height))
        images.append(img)
        labels.append(i)

ModuleNotFoundError: No module named 'cv2'

In [ ]:
images = np.array(images)
images = images.astype('float32')
images.shape

In [44]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
y = classes_df['class'].values
#print(y[:5])

y_labelencoder = LabelEncoder()
y = y_labelencoder.fit_transform(y)
print(y)

[0 0 0 ... 3 3 3]


In [48]:
from sklearn.compose import ColumnTransformer
y = y.reshape(-1,1)
ct = ColumnTransformer([('encoder', OneHotEncoder(), [0])],     remainder='passthrough')
Y = ct.fit_transform(y)
Y.shape

(6400, 4)

In [70]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images,Y = shuffle(Y, images, random_state=1)
train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.05, random_state=415)

#inspecting the shape of the training and testing
print(test_x)


  (0, 2)	1.0
  (1, 3)	1.0
  (2, 0)	1.0
  (3, 2)	1.0
  (4, 3)	1.0
  (5, 2)	1.0
  (6, 3)	1.0
  (7, 3)	1.0
  (8, 2)	1.0
  (9, 0)	1.0
  (10, 2)	1.0
  (11, 2)	1.0
  (12, 2)	1.0
  (13, 2)	1.0
  (14, 0)	1.0
  (15, 2)	1.0
  (16, 2)	1.0
  (17, 2)	1.0
  (18, 3)	1.0
  (19, 2)	1.0
  (20, 2)	1.0
  (21, 3)	1.0
  (22, 2)	1.0
  (23, 2)	1.0
  (24, 2)	1.0
  :	:
  (295, 3)	1.0
  (296, 0)	1.0
  (297, 3)	1.0
  (298, 0)	1.0
  (299, 2)	1.0
  (300, 3)	1.0
  (301, 2)	1.0
  (302, 2)	1.0
  (303, 2)	1.0
  (304, 3)	1.0
  (305, 0)	1.0
  (306, 2)	1.0
  (307, 2)	1.0
  (308, 2)	1.0
  (309, 2)	1.0
  (310, 2)	1.0
  (311, 3)	1.0
  (312, 2)	1.0
  (313, 3)	1.0
  (314, 0)	1.0
  (315, 0)	1.0
  (316, 3)	1.0
  (317, 3)	1.0
  (318, 2)	1.0
  (319, 3)	1.0


In [1]:
model.fit(train_x, train_y, epochs=300)

NameError: name 'model' is not defined

In [33]:
#pip install wget

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=c3b8d97c7ba78ff3b96e74d526212f2532c4fdb017c9777ee8379f6e0ad5bd7a
  Stored in directory: c:\users\ishno\appdata\local\pip\cache\wheels\8b\f1\7f\5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [34]:
#import wget
#url = 'https://www.kaggle.com/datasets/tourist55/alzheimers-dataset-4-class-of-images?resource=download'
#project = wget.download(url)

-1 / unknown